In [ ]:
import os 
import random 
import numpy as np
import pandas as pd 
import tensorflow as tf 
from pprint import pprint 
from pylab import plt,mpl 
plt.style.use('seaborn-v0_8')
mpl.rcParams['savefig.dpi']=300
mpl.rcParams['font.family'] ='serif'
pd.set_option('display.precision',4)
np.set_printoptions(suppress=True,precision=4)
os.environ['PYTHONHASHSEED'] = '0'
import numpy as np 
from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.layers import Dense

def transform(X):
  y = 0.05 * X**2 + 0.2 * X + np.sin(X) + 5
  y += np.random.standard_normal(len(X)) * 0.2
  return y

X = np.linspace(-2*np.pi,2*np.pi,500)
a = transform(X)

plt.figure(figsize=(10,6))
plt.plot(X,a)

a = a.reshape((len(a),-1))
print(a[:5])
lags = 5
g = tf.keras.preprocessing.sequence.TimeseriesGenerator(a,a,length=lags,batch_size=5)
print(g)

model = Sequential()
model.add(SimpleRNN(500,activation='relu',input_shape=(lags,1)))
model.add(Dense(1,activation='linear'))
model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
print(model.summary())
%%time 
model.fit(g,epochs=500,steps_per_epoch=10,verbose=False)
X = np.linspace(-6*np.pi,6*np.pi,1000)
d = transform(X)
g_ = tf.keras.preprocessing.sequence.TimeseriesGenerator(d,d,length=lags,batch_size=len(d))
f = list(g_)[0][0].reshape(len(d)-lags,lags,1)
y = model.predict(f,verbose=False)
plt.figure(figsize=(10,6))
plt.plot(X[lags:],d[lags:],label='data',alpha=0.75)
plt.plot(X[lags:],y,'r.',label='pred',ms=3)
plt.axvline(-2*np.pi,c='g',ls='--')
plt.axvline(2*np.pi,c='g',ls='--')
plt.text(-15,22,'out-of-sample')
plt.text(-2,22,'in-sample')
plt.text(10,22,'out-of-sample')
plt.legend()

In [ ]:
import pandas as pd 
import tensorflow as tf
import numpy as np

import ssl
ssl._create_default_https_context = ssl._create_unverified_context
url = 'http://hilpisch.com/aiif_eikon_id_eur_usd.csv'
symbol='EUR_USD'
raw = pd.read_csv(url,index_col=0,parse_dates=True)

def generate_data():
  data = pd.DataFrame(raw['CLOSE'])
  data.columns=[symbol]
  data = data.resample('30min',label='right').last().ffill()
  return data 

data = generate_data()
data = (data-data.mean())/data.std()
p = data[symbol].values
p = p.reshape((len(p),-1))

lags = 5
g = tf.keras.preprocessing.sequence.TimeseriesGenerator(p,p,length=lags,batch_size=5)

from keras.models import Sequential
from keras.layers import SimpleRNN ,Dense,LSTM

def create_rnn_model(hu=100,lags=lags,layer='SimpleRNN',features=1,algorithm='estimation'):
  model=Sequential()
  if layer is 'SimpleRNN':
    model.add(SimpleRNN(hu,activation='relu',input_shape=(lags,features)))

  else:
    model.add(LSTM(hu,activation='relu',input_shape=(lags,features)))
  
  if algorithm == 'estimation':
    model.add(Dense(1,activation='linear'))
    model.compile(optimizer='adam',loss='mse',metrics=['mae'])

  else:
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

  return model


model = create_rnn_model() 
%%time
model.fit(g,epochs=500,steps_per_epoch=10,verbose=False)
y = model.predict(g,verbose=False)
data['pred']=np.nan
data['pred'].iloc[lags:] = y.flatten()
data[[symbol,'pred']].plot(figsize=(10,6),style=['b','r-.'],alpha=0.75)

data[[symbol,'pred']].iloc[50:100].plot(figsize=(10,6),style=['b','r-.'],alpha=0.75)